# The intuition behind Xgboost (Extreme Gradient Boosting)

Xgboost is a supervised machine learning algorithm which, as most other ML algorithms, can be constructed as a regressor or a classifier (Chen 2016). Unsurprisingly we are gonna use it as a classifier here, but should we want to estimate conflict magnitude or fatalities instead of probabilities we could use the regressor. It is a quit extensive algorithm with many details and particularities. However, understanding it in depth is not the prime goal right now. Instead you should aim to understand the generel intuition - not least regarding concepts such as "similarity score", "gain" and "output" along with parameters such as $\lambda$ and $\gamma$. The text below only gives an very shallow overview and does not go into depth with these subjects. As such (when time permits) you should consult Chen (2016). I will also highly recommend to the videos from [Statquest](https://www.youtube.com/watch?v=OtD8wVaFm6E&t=149s) on the subject - or at least the first two. Here , I will focus on 3 characteristics of the Xgboost classifier; it is a *boosting* algorithm, it consists of *regression trees* and it is *self-regularizing*.


Different kinds of boosting is used in a lot of different machine learning algorithms. In its essence, boosting is an ensemble technique where we aggregate a lot of weak classifiers to create a single strong classifier. What set boosting algorithms apart is that they use past result to improve future predictions. In the case of predictions, we often start with a uniform predictions for all observations. Say 0.5. All observations have equal probability of being an event or an non-event. Now we go through the first round of training and creates new predictions. These new predictions will (hopefully) vary. Some will be very far from the true class other will be close. We now go through a second round of training, but this time observations which we predicted well after the first round will carry less weight while observations which we were bad at predicting will carry more weight. This procedure is iterated until no improvement is achieved or until some predefined criteria is met. In the end all models are used in a voting ensemble, usually weighted by their performance (Friedman et al. 338-339,2001). Such procedure ensures a continuing focus on "hard to classify" observations, and as such it is a particularly useful approach when working with rare events.


Turning to the individual underlying classifiers Xgboost utilizes a kind of regression trees. It might be surprising that a classifier uses regression trees and not classification tress, but to put it briefly: The reason the Xgboost algorithm uses regression trees rather then decision trees, even for classification, is that each tree produces a continues score as oppose to a binary classification at the end of each branch. These score holds more information then a hard classifications and they can be added together across all trees before being normalized to probabilities and used for classification (Chen 786, 2016). The regression trees are used to partition our data according to the split-points which best sort our data according to some measure - often "gain" og "gini" (Chen 786, 2016). While Xgboost is slightly more involved you should recognise the general logic from what you know about decision trees and random forest given Friedman et al. (2001). We keep splitting partitions until we are unable to achieve any more "gain" (Chen 786, 2016). One crucial aspect regarding this procedure, is that each split (apart from the very first) effectively constitutes an interaction. Thus the Xgboost algorithm is able to automatically identify and generate the most salient interactions.

Another important feature, specific to the regression trees used in Xgboost, is that the algorithm is self-regularizing, thus mitigating overfitting. Specifically the algorithm penalizes complex tree structures (This is where $\lambda$ and $\gamma$ comes into play). When the algorithm decides whether to create a new split, it compares the potential improvement in prediction power to the added complexity. If the added complexity is to great compared to the improvement in prediction power the split is not effectuated (Chen 787-791, 2016). Thus, the regression trees used in the boosting effort searches for relevant patterns in the data, but automatically stops before overfitting commence.

Since the Xgboost algorithm is based on tree structures, similar to the Random Forest, we can readily asses which splits generate most prediction power and thus which features are most *important* in regards to the combined prediction effort. Importance here meaning how much prediction power the specific feature contribute with, compared the the other features (Chen 787-788, 2016). As such, feature importance should be understood as a relative measure comparing how much information a specific features bring the the effort compared to the other (Friedman et al. 367-368, 2001). As you will see below, three different importance measures can readily be assessed given some trained Xgboost model. While these importance measures tells us nothing about causation, they are still highly relevant for guiding future prediction frameworks, informing theoretical debates pertaining to explanations and for formulation of concrete policy recommendations.


Naturally, this is a very superficial introduction to the Xgboost algorithm. There are a large number of other regularization strategies and mechanism which also plays a part in the actual implementation of Xgboost (Chen 787, 2016). Nevertheless this short introduction serves to illustrate why this algorithm is well-suited for conflict prediction. The boosting element makes the algorithm suitable for rare events and the fact that is is based on regressions trees both allows for automatic generation of interactions and for assessment of feature importance. Importantly, the specific trees used in this algorithm are self-regularizing thus discouraging overfitting.

# 5A.0

Import pandas as pd, matplotlib.pyplot as plt, xgboost as xgb, pickle and time. from xgboost import XGBClassifier and from sklearn import metrics

Now, the next step is depending on wheter you run Xgboost in you geo_env of have created a seperate xgb_env. If you use geo_env just load you *train_set_featureEng* and *val_set_featureEng* pickles. Then split them up (and drop some features from X) using the same appraoch as you did in the beginning of 4B.1.

In [ ]:
# pkl_file = open('train_set_featureEng.pkl', 'rb')
# train_setFE = pickle.load(pkl_file)
# pkl_file.close()

# pkl_file = open('val_set_featureEng.pkl', 'rb')
# val_setFE = pickle.load(pkl_file)
# pkl_file.close()

# y_train = train_setFE['binary_best_t1']
# X_train = train_setFE.drop(['gid','gwno','year','geometry', 'binary_best_t1', 'binary_best_t2', 'binary_best_t3', 'binary_best_t4', 'binary_best_t5'], axis =1)

# y_val = val_setFE['binary_best_t1']
# X_val = val_setFE.drop(['gid','gwno','year','geometry', 'binary_best_t1', 'binary_best_t2', 'binary_best_t3', 'binary_best_t4', 'binary_best_t5'], axis =1)

If you have created a new specifc xgb_env you should have run the week11split.ipynb script by now. If this is the case, you can just load your pickled y_train, X_train, y_val, X_val (Which are now not geopandas dataframes but just pandas dataframes)

In [ ]:
# pkl_file = open('y_trainXGB.pkl', 'rb')
# y_train = pickle.load(pkl_file)
# pkl_file.close()

# pkl_file = open('X_trainXGB.pkl', 'rb')
# X_train = pickle.load(pkl_file)
# pkl_file.close()

# pkl_file = open('y_valXGB.pkl', 'rb')
# y_val = pickle.load(pkl_file)
# pkl_file.close()

# pkl_file = open('X_valXGB.pkl', 'rb')
# X_val = pickle.load(pkl_file)
# pkl_file.close()

# 5A.1

Now train/fit and xgboost model on your training data. The approach is exactly the same as for scikit learn models. Start by only  setting the random_state of your model (e.g. to 42). No other parameters should be set now. We'll just run with the default settings for now.

In [ ]:
# fit model no training data
model = XGBClassifier(random_state = ...)
model.fit(..., ...)

Now print the model. This will allow you to inspect the default (hyper) parameter settings (it might already be printed above, just do it anyway)

In [ ]:
print(...)

Now get the predicted probabilities (predict_proba) in-sample (y_train_pred from X_train) and out-of-sample (y_val_pred from X_val). Agian, the procedure is similar to that of scikit learn so you can seek inspiration in past exercises, e.g. 3B.1

In [ ]:
y_train_pred = ...
y_val_pred = ...

Now, as you have done before print the AUC, AP and BS for both traning predictions and validation predicitons (Again 3B.1 or similar might serve as guidance)

Now plot the importance of the features. This can be done the same way as we did it with Random Forest in the beginning of 4B.2. The only thing you need to note is that you model is no longer called `rf` but `model`

Nice. However the Xgboost package has a much easier and more flexibly inbuilt methode for plotting importance. Try running the cell below and read the description:

In [ ]:
xgb.plot_importance?

The most important thing you should note from above is the three different `importance_type`: "weight", "gain" and "cover". Lest create a plot for each type below. 

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
xgb.plot_importance(model, importance_type = ...)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
xgb.plot_importance(model, importance_type = ...)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
xgb.plot_importance(model, importance_type = ...)
plt.show()


Now lets try to visualise a regression tree. If you have create a dedicated Xgb_env and followed my instructions on installing graphviz here, this should pose no problem. If graphviz refuses to work don't worry about it and just skip it. Note the tha default tree depth is 6 which makes the plot quit hard to make out in jupyter. As such, we also save it as a PDF "tree.pdf". Go have a look at it when it is done (it is in the same folder as this script).

In [ ]:
plt.rcParams['figure.figsize'] = [100, 15]
xgb.plot_tree(model,num_trees=0)
plt.savefig("tree.pdf")
plt.show()

# 5A.2

Alright. We will now start tweaking the parameters of our Xgboost model. Start by running the code below and browse the different parameters:

In [ ]:
XGBClassifier?

So, there is a lot here. We will only play with a subset but feel free to experiment at the end of the assigment. For now, run a new model but this time set n_estimators = 70, max_depth = 3, learning_rate = 0.2, gamma = 0.5, reg_lambda = 3 and random_state = 42.

In [ ]:
model2 = XGBClassifier(..., ..., ..., ..., ..., ...)
model2.fit(..., ...)

Print the model (even if it printed itself above)

Create y_train_pred and y_val_pred using `predict_proba` (as you have did above and have done in previous exercises)

Print the AUC, AP and BS of the training and validation predictions

Hmmm. These might be a bit worse or a bit better then above. We could try again with new parameters an see if we can do better. As you might expect we need to do this a lot of times if we want to find the optimal combinations of parameters. As such it is a task we should automatize. This can be done in a pretty simple for-loop, but even more simple by using some tools from the scikit learn library

# 5A.3

Iterating over multible different combinations of parameters is called *Grid Search* - or *Random Search* if you don't look at the whole "grid" of combinations. You can get more inspiration here https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost and here https://www.datacamp.com/community/tutorials/xgboost-in-python

Lest try both approaches (grid and random):

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

Now we create a dictionary containing different ranges of parameters we want to iterate over. If you read Chen (2016) or watch the Statquest videos recommended the parameters should be familiar to you.

In [ ]:
params = {
        'max_depth': [4, 5, 6, 7],
        'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
        'gamma': [0.05, 0.1, 0.5, 1, 1.5],
        'reg_lambda': [0.01, 0.05, 0.1, 0.5]
        }

Now create model3 below. Note, that to make the whole thing go a bit faster I set n_estimators = 32. When you are done with the assigment, before uploading, I recommend you set it higher, say 64 or 128, and let it run over night. (Other than this, you don't need change any code below)

In [ ]:
model3 = XGBClassifier(n_estimators=16)

We will start with a "random" search (RandomizedSearchCV). So we here tell the algorithm how many times it should run (param_comb) and it just try out random combinations of the parameter ranges we specificed in the dictionary above. I've intially set it to 8 for speed but you should try setting it higher (say 16) when you have been thorugh the exercise once.  

Each model is evaluated via [k-fold cross validation](https://machinelearningmastery.com/k-fold-cross-validation/). That is, a kind of validation where we split the training set and uses some parts of it for training and some part of it for validation. Specifically we set k (folds) at 4, so we use 3/4 of the data to predict the last 1/4. As such the logic is the same as normally, here we just only use the traning set while iterating and what is traning and validation changes constantly. We can of course use our usual validation set in the end to asses the final result (no code needs changing). Note that I have set the scoring criteria to "average_precision" as we know this is most prudent given the nature of our data. Now, this will take some time so start it and go get yourselves a good cuppa.

In [ ]:
folds = 4
param_comb = 8

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(model3, param_distributions=params, n_iter=param_comb, scoring='average_precision', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3, random_state=42 )

start_time = time.time()
random_search.fit(X_train,y_train)
end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')

Now, lest print the best combination of parameters which our random search stumbled upon:

In [ ]:

print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n')


print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n')

print('\n Best hyperparameters (this is the most important one):')
print(random_search.best_params_)
print('\n')


So, we can run a model with these new parameters either by explicitly writing declaring them or by getting them from the new dictionary (random_search.best_params_). We here do the later as it is less prone to error.

In [ ]:
model4 = XGBClassifier(n_estimators = 10, 
                       reg_lambda = random_search.best_params_['reg_lambda'],
                       max_depth = random_search.best_params_['max_depth'], 
                       learning_rate = random_search.best_params_['learning_rate'],
                       gamma =  random_search.best_params_['gamma'], 
                       random_state = 42)


Fit model4:

In [ ]:
model4.fit(..., ...)

Get the predicitions:

And print the AUC, AP and BS for both train and validation (you know this by now):

Hopefuly the results are a bit better then when we started out - if not it might be because we run with a quit low number of estimatiors for speed. Go back an change that when you are doen with the exercise.

Lets try the grid search now (GridSearchCV). The difference here is that we now try all possible combinations given the parameter ranges in our params dictionary. As such this is a more exhaustive search for good parameters compared to the random search. However, it is of course also more time consuming. **In reality it migth take days if not weeks**. So make sure you have a low number of estimators and maybe create a more sparse params dictionary - that is remove some of the options to decrease the number of combinations. (Of course Be aware that you don't remove any values found good be the random search).

In [ ]:
grid = GridSearchCV(estimator=model3, param_grid=params, scoring='average_precision', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3 )

start_time = time.time()
grid.fit(X_train,y_train)
end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')

In [ ]:

print('\n Best estimator:')
print(grid.best_estimator_)
print('\n')

print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(grid.best_score_ * 2 - 1)
print('\n')

print('\n Best hyperparameters:')
print(grid.best_params_)
print('\n')

Now, train  a new model (model5) with the new parameters (remember to takethe new best estimates from grid.best_params_ and not random_search.best_params_). Create the predictions and print the AUC, AP and BS for both training and validation set.

In [ ]:
model5 = XGBClassifier(n_estimators = 10, 
                       reg_lambda = ...,
                       max_depth = ..., 
                       learning_rate = ...,
                       gamma =  ..., 
                       random_state = 42)

model5.fit(X_train, y_train)

And you are done! As you have probably run the models with a pretty low n_estimators i suggest you go back and change this. Maybe to 64 or 128 depeding on how fast your computer is and try to run the random grid search again (the full grid search will take weeks finishing under such conditions so don't bother). Let the new random search run over night and look at your results in the morning. Cheers!